In [9]:
import os
from pathlib import Path
from datasets import load_dataset


local_dir =  "data" 
DATA_CACHE_DIR = os.path.join(Path(), local_dir)
os.makedirs(DATA_CACHE_DIR, exist_ok=True)

# download the dataset
dataset = load_dataset("HuggingFaceFW/fineweb-edu", "sample-10BT", split="train", cache_dir=DATA_CACHE_DIR)

# tokenize the dataset
tokenizer = tiktoken.get_encoding("gpt2")

enc = tiktoken.get_encoding("gpt2")
eot = enc._special_tokens['<|endoftext|>']

README.md:   0%|          | 0.00/25.5k [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/2110 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/98 [00:00<?, ?it/s]

In [29]:
len(list(enumerate(dataset[0:2]['text'])))

2

In [80]:
SHARD_SIZE = 10e8
output_dir = "shards"


def tokenize(docs_per_cpu, n):
    import os
    from pathlib import Path
    from datasets import load_dataset
    import tiktoken
    
    # set up tokenizer
    enc = tiktoken.get_encoding("gpt2")
    eot = enc._special_tokens['<|endoftext|>']
    tokenizer = tiktoken.get_encoding("gpt2")
    
    # load the dataset
    dataset = load_dataset("HuggingFaceFW/fineweb-edu", "sample-10BT", split="train", cache_dir=os.path.join(Path(), "data"))

    tokens = [eot]
    docs_processed = 0
    start = n*docs_per_cpu
    if len(dataset) - (n*docs_per_cpu) < docs_per_cpu:
        end = len(dataset)
    else:
        end = start + docs_per_cpu
    for idx, d in enumerate(dataset[start:end]['text']):
        #print(f"{idx:,}", end="\r")
        tokens.extend(enc.encode_ordinary(d))
        docs_processed += 1
    return np.array([len(tokens)], dtype=np.uint32), docs_processed

tokenize(300, 0)


Resolving data files:   0%|          | 0/2110 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/98 [00:00<?, ?it/s]

(array([277581], dtype=uint32), 300)

In [82]:
import time
import math
from functools import partial

import multiprocessing
#multiprocessing.set_start_method('fork')   
import concurrent.futures as cf

num_cpus = os.cpu_count()
total_docs = len(dataset)
docs_per_cpu = math.ceil(total_docs/num_cpus)
print(f"""
dataset statistics
------------------
documents: {total_docs:,}
docs_per_cpu: {docs_per_cpu}""")


def count_tokens(dataset, docs_per_cpu, n):
    total_length = 0
    docs_processed = 0
    start = n*docs_per_cpu
    if len(dataset) - (n*docs_per_cpu) < docs_per_cpu:
        end = len(dataset)
    else:
        end = start + docs_per_cpu
    for idx, d in enumerate(dataset[start:end]['text']):
        #print(f"{idx:,}", end="\r")
        total_length += len(d)
        docs_processed += 1
    return total_length, docs_processed

f = partial(count_tokens, dataset, docs_per_cpu)
t = partial(tokenize, 10)

start = time.time()
with cf.ProcessPoolExecutor(max_workers = num_cpus) as ex:
    results = list(ex.map(t, range(num_cpus)))
print(results)
print(f"Processed documents in {time.time()-start} seconds")
assert(sum([r[1] for r in results]) == total_docs)

    
avg_doc_len = sum([r[0] for r in results]) / total_docs
print(f"avg doc length: {avg_doc_len:,} characters")




dataset statistics
------------------
documents: 9,672,101
docs_per_cpu: 322404
[(array([9449], dtype=uint32), 10), (array([6019], dtype=uint32), 10), (array([7444], dtype=uint32), 10), (array([9343], dtype=uint32), 10), (array([8439], dtype=uint32), 10), (array([6340], dtype=uint32), 10), (array([9240], dtype=uint32), 10), (array([7958], dtype=uint32), 10), (array([7276], dtype=uint32), 10), (array([8285], dtype=uint32), 10), (array([5532], dtype=uint32), 10), (array([9341], dtype=uint32), 10), (array([4791], dtype=uint32), 10), (array([6091], dtype=uint32), 10), (array([9425], dtype=uint32), 10), (array([6381], dtype=uint32), 10), (array([7728], dtype=uint32), 10), (array([7410], dtype=uint32), 10), (array([7965], dtype=uint32), 10), (array([23120], dtype=uint32), 10), (array([7757], dtype=uint32), 10), (array([29517], dtype=uint32), 10), (array([10627], dtype=uint32), 10), (array([8823], dtype=uint32), 10), (array([5791], dtype=uint32), 10), (array([14111], dtype=uint32), 10), (arr

AssertionError: 